In [17]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.cross_validation import cross_val_score, train_test_split, KFold
from sklearn import metrics, preprocessing
import json
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn import decomposition

In [18]:
movies = pd.read_csv("tmdb_5000_movies.csv")
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [69]:
credits = pd.read_csv("tmdb_5000_credits.csv")
credits = credits.drop(['title'], axis=1)
credits.head()

,movie_id,cast,crew
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [70]:
movies = movies.rename(columns={'id':'movie_id'})
tmdb = pd.merge(movies, credits, on=['movie_id'])
tmdb.head()

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [71]:
print("Jumlah row pada data: {}".format(tmdb.shape[0]))
print("Jumlah feature pada data: {}".format(tmdb.shape[1]))

Jumlah row pada data: 4803
Jumlah feature pada data: 22


In [72]:
tmdb.describe()

,budget,movie_id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [73]:
tmdb.isnull().sum()

budget                     0
genres                     0
homepage                3091
movie_id                   0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
dtype: int64

In [74]:
for col in tmdb.columns:
    if(tmdb[col].dtype in ['int64', 'float64']):
        median = tmdb[col].median()
        tmdb[col].replace(to_replace=0, value=median, inplace=True)
        tmdb[col].fillna(median, inplace=True)
    else:
        modus = tmdb[col].mode()
        tmdb[col].fillna(str(modus), inplace=True)

In [75]:
tmdb.isnull().sum()

budget                  0
genres                  0
homepage                0
movie_id                0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
cast                    0
crew                    0
dtype: int64

In [88]:
company = tmdb['production_companies']
genre = tmdb['genres']
keyword = tmdb['keywords']
prod_country = tmdb['production_countries']
spoken = tmdb['spoken_languages']
cast = tmdb['cast']
crew = tmdb['crew']

def convert_multivalue(attribute):
    ls = []
    for i in attribute:
        v = json.loads(i)
        if len(v) != 0:
            ls.append(v[0].get('name'))
        else:
            ls.append('None')
    
    return ls

tmdb['production_companies'] = convert_multivalue(company)
tmdb['genres'] = convert_multivalue(genre)
tmdb['keywords'] = convert_multivalue(keyword)
tmdb['production_countries'] = convert_multivalue(prod_country)
tmdb['spoken_languages'] = convert_multivalue(spoken)
tmdb['cast'] = convert_multivalue(cast)
tmdb['crew'] = convert_multivalue(crew)

In [89]:
tmdb.head()

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,Action,http://www.avatarmovie.com/,19995,culture clash,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners,...,2787965087,162.0,English,Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington,Stephen E. Rivkin
1,300000000,Adventure,http://disney.go.com/disneypictures/pirates/,285,ocean,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney Pictures,...,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp,Dariusz Wolski
2,245000000,Action,http://www.sonypictures.com/movies/spectre/,206647,spy,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Columbia Pictures,...,880674609,148.0,Français,Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig,Thomas Newman
3,250000000,Action,http://www.thedarkknightrises.com/,49026,dc comics,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Legendary Pictures,...,1084939099,165.0,English,Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale,Hans Zimmer
4,260000000,Action,http://movies.disney.com/john-carter,49529,based on novel,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Walt Disney Pictures,...,284139100,132.0,English,Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch,Andrew Stanton


In [90]:
tmdb.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

In [91]:
tmdb.corr(method='spearman').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

In [67]:
# for i in tmdb['production_companies']:
#     prod_name = set(i)
prod_name = set(tmdb['production_companies'])

In [68]:
print(prod_name)

{'Copacetic Pictures', 'Walt Disney', 'Appian Way', 'Smokewood Entertainment Group', 'An Olive Branch Productions', 'Journeyman Pictures', 'Velvet Steamroller Entertainment', 'Les Productions du Trésor', 'Deviant tmdbs', 'tmdbfabriken Baltic Sea AB', 'Jafar Panahi tmdb Productions', 'TriStar Pictures', 'Act III Communications', 'Forecast Pictures', 'Propaganda tmdbs', 'Danish tmdb Institute', 'BBC', 'Eyeline Entertainment', 'Nic Arts', 'Relativity Media', 'Parallel Pictures', 'American tmdbworks', 'Cambridge Productions', 'Max TV', 'Conquest Productions', 'Double Edge tmdbs', 'Handsomecharlie tmdbs', 'Screen Gems', 'Triumphant Entertainment', 'Indian Paintbrush', 'Pegasus Motion Pictures', 'Lucky Coffee Productions', 'Hearst Entertainment Productions', 'Believe Entertainment', 'Bel Air Entertainment', 'Eivissa Productions', 'GK tmdbs', 'Mirage Enterprises', 'France 3 Cinéma', 'Kinokompaniya CTB', 'Troma Entertainment', 'National tmdb Board of Canada (NFB)', 'Asghar Farhadi Productions'

### One Hot Encoding

In [89]:
object_var = tmdb.select_dtypes(include=['object']).copy()
object_var.head(5)

,genres,production_companies,production_countries,spoken_languages,status
0,Action,Ingenious tmdb Partners,United States of America,English,Released
1,Adventure,Walt Disney Pictures,United States of America,English,Released
2,Action,Columbia Pictures,United Kingdom,Français,Released
3,Action,Legendary Pictures,United States of America,English,Released
4,Action,Walt Disney Pictures,United States of America,English,Released


In [77]:
object_var.shape

(4803, 5)

In [90]:
object_var.columns

Index(['genres', 'production_companies', 'production_countries',
       'spoken_languages', 'status'],
      dtype='object')

In [91]:
numeric_var = tmdb.select_dtypes(include=['int64', 'float64']).copy()
numeric_var.head(5)

,budget,id,popularity,revenue,runtime,vote_average,vote_count
0,237000000,19995,150.437577,2787965087,162.0,7.2,11800
1,300000000,285,139.082615,961000000,169.0,6.9,4500
2,245000000,206647,107.376788,880674609,148.0,6.3,4466
3,250000000,49026,112.312950,1084939099,165.0,7.6,9106
4,260000000,49529,43.926995,284139100,132.0,6.1,2124


In [92]:
for col in object_var.columns:
    object_var = pd.get_dummies(object_var, columns=[col])
    
tmdb_encoded = pd.concat([object_var, numeric_var], axis=1)

In [93]:
tmdb_encoded.head()

,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_Foreign,...,status_Post Production,status_Released,status_Rumored,budget,id,popularity,revenue,runtime,vote_average,vote_count
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,237000000,19995,150.437577,2787965087,162.0,7.2,11800
1,0,1,0,0,0,0,0,0,0,0,...,0,1,0,300000000,285,139.082615,961000000,169.0,6.9,4500
2,1,0,0,0,0,0,0,0,0,0,...,0,1,0,245000000,206647,107.376788,880674609,148.0,6.3,4466
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,250000000,49026,112.312950,1084939099,165.0,7.6,9106
4,1,0,0,0,0,0,0,0,0,0,...,0,1,0,260000000,49529,43.926995,284139100,132.0,6.1,2124


### PCA and Data Splitting

In [169]:
tmdb_encoded_cols = tmdb_encoded.columns
train_cols = [x for x in tmdb_encoded_cols if x != 'revenue']

X = tmdb_encoded[train_cols]
y = tmdb_encoded['revenue']

pca = decomposition.PCA(n_components=1).fit(X)
X_pca = pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, random_state=91, test_size=0.3,
)

In [170]:
print(X_pca.shape)
print(X_train.shape)
print(X_test.shape)

(4803, 1)
(3362, 1)
(1441, 1)


In [145]:
y_train

81      758539785
113     938212738
1048    175372502
3753     44349000
1889     21567853
1804            0
3318            0
1877     29548291
1576    114663461
3771            0
2551     39421467
266     347234916
3518     27165581
2883     12077441
4229      3566225
988      10541523
818      95000000
4537            0
3478      6230276
1296     75850624
4511     33456317
3772      2426851
1843            0
1068    104478416
3639       742898
702     176070171
2369     20275812
3631      3049135
1330    163720069
445     214918407
          ...    
1134     20915465
83              0
2056     15447154
1871    229736344
398     311312624
2126     37672350
4727            0
1341     21834845
654     128905366
198      61648500
3371            0
2557     30226144
4753            0
1501     50041732
4438            0
1832    152500343
3732            0
1530    161772375
984      44434439
1169     95020213
913     225990978
4597            0
156     456758981
4738      3221152
1828     7

### Decision Tree Regression

In [171]:
dt_reg = DecisionTreeRegressor(max_depth=2)
dt_reg.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [172]:
y_pred = dt_reg.predict(X_test)
mse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("MSE: ", mse)

MSE:  122133940.826


### Linear Regression

In [166]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [125]:
y_pred = lin_reg.predict(X_test)
# mse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
# print("MSE: ", mse)

# The coefficients
print('Coefficients: \n', lin_reg.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

# Plot outputs
plt.scatter(X_test, diabetes_y_test,  color='black')
plt.plot(X_test, diabetes_y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

Coefficients: 
 [  2.91097378e+00   2.23142429e+01   6.92295390e+04  -2.36830605e+04
   2.94726482e+05   2.93526675e+06  -5.05821468e+06   1.45905837e+04
  -1.69597325e+07  -3.66488001e+05  -2.07591676e+07   1.69490126e+07
  -1.54961599e+07  -8.17658802e+06   8.48910430e+06   1.98281714e+07
  -1.18565866e+07   7.36961761e+06  -9.47953812e+05  -8.58550596e+06
   1.87512653e+06  -1.22533864e+06   2.73482261e+07   4.69340291e+06
  -2.01070900e+07   3.03073648e+07   1.08933835e+06  -2.21879191e+06
   1.01330375e+07   3.95644134e+06  -1.77892326e+07   1.62518102e+07
  -8.01808598e+06  -1.77822863e+07  -4.16687728e+06   1.56719975e+07
   4.72752009e+06   7.07301003e+05  -6.90709280e+06   4.04845632e+07
   2.30090392e+07   1.29065901e+07  -2.13266989e+07   3.55475683e+06
  -2.55540142e+06  -3.51258666e+07  -1.19205905e+07  -7.09550129e+06
   2.22926404e+06   2.40885498e+07   1.33689504e+07   4.99454974e+06
  -6.36941502e+06  -6.00880163e+06  -1.16100127e+07  -5.20517818e+06
  -6.01215462e+06 

NameError: name 'mean_squared_error' is not defined

### Logistic Regression

In [122]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [123]:
y_pred = log_reg.predict(X_test)
mse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("MSE: ", mse)

MSE:  364538617.346


### Decision Tree Classification

In [119]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [121]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Akurasi: ", accuracy,"%")

Akurasi:  19.6391394865 %


In [42]:
confusion_matrix(y_test, y_pred)

array([[271,   0,   0, ...,   0,   0,   0],
       [  1,   0,   0, ...,   0,   0,   0],
       [  1,   0,   0, ...,   0,   0,   0],
       ..., 
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])